In [5]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 285kB/s ta 0:00:01


In [3]:
!pip install geocoder

    100% |████████████████████████████████| 71kB 322kB/s ta 0:00:01


In [1]:
!pip install geopy

    100% |████████████████████████████████| 71kB 295kB/s ta 0:00:01


In [6]:
# Import pandas
import pandas as pd
import geocoder as gc
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

# Import BoxPlot, output_notebook, and show from bokeh.charts
from bokeh.charts import BoxPlot, Donut, Bar, Histogram, output_notebook, show
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.layouts import gridplot, row
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import ColumnDataSource


In [7]:
# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

['Sheet1']


In [8]:
# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [9]:
df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina'],
      dtype='object')

In [10]:
df.shape

(4842, 10)

In [11]:
#Cópia por valores, não mexer no DF principal
disciplinas = df.copy()

disciplinas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4842 entries, 0 to 4841
Data columns (total 10 columns):
a_ID                  4842 non-null int64
CEP                   4842 non-null int64
ano_ingresso          4842 non-null int64
periodo_ingresso      4842 non-null int64
status                4842 non-null object
ano_disciplina        4842 non-null int64
periodo_disciplina    4842 non-null int64
nota                  4842 non-null float64
disciplina_ID         4842 non-null int64
status.disciplina     4842 non-null object
dtypes: float64(1), int64(7), object(2)
memory usage: 416.1+ KB


In [12]:
disciplinas.columns.values.tolist()

['a_ID',
 'CEP',
 'ano_ingresso',
 'periodo_ingresso',
 'status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [13]:
#Retira informações desnecessárias para essa análise

disciplinas.drop(disciplinas.columns[0:4], axis=1, inplace=True)

In [14]:
#Objetivo 1 -> Média da disciplina X por período e ano
#Objetivo 2 -> Taxa de trancamento e cancelamento por período e ano
disciplinas.columns.values.tolist()

['status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [15]:
disciplinas = disciplinas[disciplinas['status'] == 'ATIVO']
disciplinas.head()

,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
3,ATIVO,2014,2,6.1,0,Aprovado
4,ATIVO,2014,1,3.0,0,Reprovado
5,ATIVO,2014,2,7.2,0,Aprovado
8,ATIVO,2014,2,8.9,0,Aprovado
12,ATIVO,2014,2,5.6,0,Aprovado


In [16]:
disciplinas_categorias =  disciplinas.groupby(['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']).mean()

# convert the index to a column
disciplinas_categorias.reset_index(inplace = True )
disciplinas_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
ano_disciplina        42 non-null int64
periodo_disciplina    42 non-null int64
disciplina_ID         42 non-null int64
nota                  42 non-null float64
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [17]:
disciplinas_categorias[disciplinas_categorias['disciplina_ID'] == 0]

,ano_disciplina,periodo_disciplina,disciplina_ID,nota
0,2014,1,0,5.872727
6,2014,2,0,5.625676
13,2015,1,0,6.126316
19,2015,2,0,5.486555
26,2016,1,0,4.139394
33,2016,2,0,5.750758


In [18]:
# Make a box plot: p
p = BoxPlot(disciplinas_categorias, values='nota', label='disciplina_ID', color='disciplina_ID',
             title='Distribuição da média de notas das disciplinas',
             legend='bottom_right')

# Set the y axis label
p.yaxis.axis_label='Média por período'
p.add_tools(HoverTool(tooltips=[("Nota", "$y")]))

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

In [20]:
ps = []
#tbs = []

for atual in range(0,7):
    p = BoxPlot(disciplinas_categorias[disciplinas_categorias['disciplina_ID'] == atual], values='nota', label='periodo_disciplina',
                        color='periodo_disciplina', title='Ddisciplina '+str(atual), legend='bottom_right')
    p.yaxis.axis_label='Média por período'
    p.add_tools(HoverTool(tooltips=[("Nota", "$y")]))
    
    #temp = Panel(child=p, title='Ddisciplina '+str(atual))
    
    #tbs.append(temp)
    ps.append(p)
    
# Make a box plot: p


# Set the y axis label

grid = gridplot([ps[0:2], ps[2:4], ps[4:6], [None, ps[6]]], sizing_mode='scale_width')
#grid = gridplot([[ps[0], ps[1]], [ps[2], ps[3]], [ps[4], ps[5]], [None, ps[6]]], sizing_mode='scale_width')

#tabs = Tabs(tabs=[ tbs[0], tbs[5] ],sizing_mode='scale_width' )

#show(tabs)
# show the results
show(grid)

In [21]:
##Quantidade de alunos ativos/Trancados/cancelados/...
count_series = pd.DataFrame(columns=('Status', 'Count'))
count_series["Status"] = df["status"].unique().tolist()
tam = len(df)

for atual in range(0,len(count_series)):
    count_series.iloc[atual]["Count"] = df[df["status"] == count_series.iloc[atual]["Status"]]["status"].count()

def porcent(val):
    p = (val*100)/tam
    return p
    
graph = Donut(count_series, label="Status", values="Count", title="Situação dos alunos" )
graph.add_tools(HoverTool(tooltips=[("Alunos", str(count_series.iloc[1]["Count"])+"/"+str(tam)), #falta trocar o 1 pelo número de alunos da categoria
                                    ("Porcentagem", str(porcent(1000))) #falta trocar o 1000 pelo numero de alunos da categoria
                                   ]
                         ))

show(graph)

In [23]:
###Verificar desistência
def popula(string):
    aux = []
    for atual in range (2014, 2017):
        aux.append(df[df["ano_ingresso"] == atual][df["status"] == string]["status"].count())
    return aux

count_desist = pd.DataFrame(columns=('ANO', 'CANCELADO', 'ATIVO', 'TRANCADO', 'CONCLUIDO', 'FORMANDO', 'FORMADO'))
count_desist["ANO"] = [2014, 2015, 2016]

for atual in count_desist.columns.values:
    if atual != "ANO":
        count_desist[atual] = popula(atual)

print(count_desist)



    ANO  CANCELADO  ATIVO  TRANCADO  CONCLUIDO  FORMANDO  FORMADO
0  2014        584    843       112         13        38       10
1  2015        329   1530       114          7         0        0
2  2016        134   1067        56          0         5        0


/home/danielmarx/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [24]:

BLEND = blend('CANCELADO', "ATIVO",  "TRANCADO",  "CONCLUIDO",  "FORMANDO",  "FORMADO",
              name='unidade', labels_name='Situacao')

bar = Bar(count_desist, values=BLEND,
          label=cat(columns='ANO', sort=False),
          stack=cat(columns='Situacao', sort=False),
          color="Situacao",
          legend='bottom_center',
          title="Índice de formação e desitencia de alunos por ano",
          ylabel = "Alunos",
          tooltips=[('Tipo', "$y")]) ##Corrigir isso também

output_notebook()

show(bar)

Loading BokehJS ...

In [25]:
###Verificar NOtas ENEM
ps = []
temp = ["nota", "enen-nota", "enem-matematica", "enem-redacao"]

for atual in range(0, len(temp)):
    p = BoxPlot(disciplinas_categorias, values=temp[atual], label='disciplina_ID',
                        color='disciplina_ID', title=str(temp[atual])+" x Desempenho em disciplinas ", legend='bottom_right')
    p.yaxis.axis_label='Média por período'
    p.add_tools(HoverTool(tooltips=[("Nota", "$y{f.fff}")]))
    
    #temp = Panel(child=p, title='Ddisciplina '+str(atual))
    
    #tbs.append(temp)
    ps.append(p)
    

grid = gridplot([ps[0:2], ps[2:4]], sizing_mode='scale_width')

output_notebook()
show(grid)

KeyError: 'enen-nota'

In [83]:
#Verificar distancias
distancias = df[df["status"] == "ATIVO"].copy()
distancias["LAT"], distancias["LON"], distancias["KM"] = [0,0,0]
distancias = distancias.reset_index()
uf = gc.google("59064741").latlng

for i in tqdm(range(len(distancias))): ## trocar o range por range(len(distancias))
    st = distancias.loc[i,'CEP']
    g = gc.google(st)
    if g.lat == None:
        distancias.loc[i, "LAT"] = 0
    elif g.lng == None:
        distancias.loc[i, "LON"] = 0
    else:
        distancias.loc[i, "LON"] = g.lng
        distancias.loc[i, "LAT"] = g.lat
print("Completo")

distancias.to_csv('LatLong_Alunos3.csv', encoding="utf-8")
##Salva tabela criada
    
#UFRN 59064-741
distancias.head()

100%|██████████| 3440/3440 [38:44<00:00,  1.38it/s]  

Completo


,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,0
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600,0
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,0
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,0
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,0


# teste =============================

In [84]:
distancias2 =  pd.read_csv('LatLong_Alunos.csv', encoding="utf-8", index_col=0)
distancias2 = distancias2[distancias2["a_ID"] == 3]
distancias2

,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.2066,0
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.2066,0
136,266,3,59088150,2014,1,ATIVO,2014,1,5.3,1,Aprovado,-5.872282,-35.2066,0
233,486,3,59088150,2014,1,ATIVO,2014,1,8.4,2,Aprovado,-5.872282,-35.2066,0
322,693,3,59088150,2014,1,ATIVO,2014,2,5.3,3,Aprovado,-5.872282,-35.2066,0
458,913,3,59088150,2014,1,ATIVO,2014,2,5.8,4,Aprovado,-5.872282,-35.2066,0
558,1087,3,59088150,2014,1,ATIVO,2014,2,3.4,5,Reprovado,-5.872282,-35.2066,0
559,1088,3,59088150,2014,1,ATIVO,2015,1,6.1,5,Aprovado,-5.872282,-35.2066,0
742,1406,3,59088150,2014,1,ATIVO,2014,2,5.8,6,Aprovado,-5.872282,-35.2066,0


# ============================================

In [197]:
distancias =  pd.read_csv('LatLong_Alunos.csv', encoding="utf-8", index_col=0)

for atual in tqdm(range(len(distancias))): ## trocar o range por range(len(distancias))
    lt = distancias.loc[atual, "LAT"]
    ln = distancias.loc[atual, "LON"]
    if lt != 0. and ln != 0.:
        compare = (lt, ln)
        #print(vincenty(uf, compare).km)
        distancias.loc[atual, "KM"] = vincenty(uf, compare).km
        
distancias.to_csv('LatLong_Alunos2.csv', encoding="utf-8")    
distancias.head()

100%|██████████| 3440/3440 [00:04<00:00, 742.42it/s] 


,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600,4.027258
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033


In [160]:
distancias

,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600,4.027258
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033
5,13,8,59151610,2014,1,ATIVO,2014,2,1.5,0,Reprovado,0.000000,0.000000,0.000000
6,14,8,59151610,2014,1,ATIVO,2015,1,6.9,0,Aprovado,0.000000,0.000000,0.000000
7,21,12,59547000,2014,1,ATIVO,2014,2,2.6,0,Reprovado,-5.516642,-36.067595,101.201317
8,22,12,59547000,2014,1,ATIVO,2015,1,5.9,0,Aprovado,-5.516642,-36.067595,101.201317
9,25,14,59150500,2014,1,ATIVO,2014,2,1.2,0,Reprovado,0.000000,0.000000,0.000000


## Determinar Correlação entre média de notas e distância até a UF 
### Passo 1 - Determinar Covariância -> (Σ[(xi-xmed)*(yi-ymed)])/n-1
  
### Passo 2 - Determinar Variancia de cada variável (X,Y) 
### Passo 3 - Determinar Correlação :D


### Validar valores

In [199]:
#Considera apenas as aprovações -> Validação 1
distancias = distancias[distancias["status.disciplina"] == 'Aprovado']

#Removendo distâncias desnecessárias -> Validação 2 e 3
distancias = distancias[distancias['KM'] != 0]
distancias = distancias[distancias['CEP'] != 0]

#Validação 4
distancias = distancias[distancias['KM'] < 20]

distancias

,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033
12,28,15,59151800,2014,1,ATIVO,2014,2,8.5,0,Aprovado,-5.890733,-35.196979,6.242204
14,36,20,59090620,2014,1,ATIVO,2014,2,6.4,0,Aprovado,-5.885479,-35.168988,7.204428
16,39,23,59070750,2014,2,ATIVO,2015,1,8.9,0,Aprovado,-5.833169,-35.231361,2.220069
17,41,25,59115685,2014,1,ATIVO,2016,1,7.3,0,Aprovado,-5.744671,-35.271933,12.132286
20,44,26,59290000,2014,1,ATIVO,2015,2,6.6,0,Aprovado,-5.776116,-35.368568,18.616898
21,45,27,59040240,2014,1,ATIVO,2014,1,5.9,0,Aprovado,-5.793517,-35.223136,4.891914


In [215]:
#Protótipo da média

#distancias2 = distancias.sort_values(['KM'], ascending=[False])
#distancias2 = distancias2[distancias2["a_ID"] == 3]

#media_teste =  distancias2['nota'].mean()

#distancias2 = distancias.sort_values(['KM'], ascending=[False])

#distancias2

,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
322,693,3,59088150,2014,1,ATIVO,2014,2,5.3,3,Aprovado,-5.872282,-35.2066,4.027258
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.2066,4.027258
233,486,3,59088150,2014,1,ATIVO,2014,1,8.4,2,Aprovado,-5.872282,-35.2066,4.027258
458,913,3,59088150,2014,1,ATIVO,2014,2,5.8,4,Aprovado,-5.872282,-35.2066,4.027258
136,266,3,59088150,2014,1,ATIVO,2014,1,5.3,1,Aprovado,-5.872282,-35.2066,4.027258
742,1406,3,59088150,2014,1,ATIVO,2014,2,5.8,6,Aprovado,-5.872282,-35.2066,4.027258
559,1088,3,59088150,2014,1,ATIVO,2015,1,6.1,5,Aprovado,-5.872282,-35.2066,4.027258


In [223]:
#Lista com ID dos alunos validados
alunos_validos = distancias.a_ID.unique()
alunos_validos

array([  2,   3,   5,   7,  15,  20,  23,  25,  26,  27,  30,  31,  33,
        38,  39,  41,  47,  54,  55,  57,  60,  61,  62,  63,  64,  70,
        71,  72,  80,  83,  86,  87,  93,  95,  98, 103, 108, 110, 111,
       112, 117, 120, 123, 125, 128, 129, 130, 132, 146, 153, 154, 157,
       158, 160, 162, 165, 166,  37, 188, 196, 197, 137, 139, 238, 240,
       242, 243, 245, 257, 265, 267, 272, 275, 276, 277, 279, 280, 281,
       282, 285, 286, 287, 289, 290, 292, 295, 297, 300, 301, 303, 306,
       307, 309, 315, 316, 318, 321, 322, 324, 326, 327, 328, 329, 334,
       338, 342, 344, 346, 347, 348, 351, 354, 355, 356, 359, 361, 363,
       365, 366, 383, 386, 387, 389, 391, 395, 396, 397, 398, 403, 405,
       406, 408, 409, 410, 411, 412, 414, 417, 424, 426, 430, 431, 434,
       437, 438, 445, 447, 448, 450, 454, 455, 456, 457, 458, 459, 260,
       461, 463, 464, 466, 468, 470, 475, 304, 312, 313, 314, 481, 317,
       323, 330, 489, 333, 492, 494, 495, 353, 368, 373, 374, 37

### Ocorrência dos dados

In [224]:
#Valores 'chave' da análise
valor_x_list = []
valor_y_list = []
for i in range(len(alunos_validos)):
    
    #Seleciona todas as ocorrências do aluno com aquele ID
    aluno = distancias[distancias["a_ID"] == alunos_validos[i]]    
    
    #Calcula média das notas para aquele aluno
    media =  aluno['nota'].mean()
    distancia_UF = aluno['KM'].mean()
    
    #Adiciona resultado à lista de distribuição X 
    if( media <= 6.0 ):
        valor_x_list.append(0)
    elif( media > 6.0 and media <= 7.0 ):
        valor_x_list.append(1)
    elif( media > 7.0 and media <= 8.0 ):
        valor_x_list.append(2)
    else:
        valor_x_list.append(3)
        
    #Adiciona resultado à lista de distribuição Y 
    if( distancia_UF <= 1.5 ):
        valor_y_list.append(0)
    elif( distancia_UF > 1.5 and distancia_UF <= 4.0 ):
        valor_y_list.append(1)
    elif( distancia_UF > 4.0 and distancia_UF <= 8.0 ):
        valor_y_list.append(2)
    else:
        valor_y_list.append(3)

## Tabela de Ocorrências

In [252]:
#Tabela de Ocorrências
distribuicao = pd.DataFrame(columns=('Valor X_Nota', 'Valor Y_Distancia', 'Xi - Xmed', 'Yi - Ymed'
                                     , 'Prod', '(Xi - Xmed)^2', '(Yi - Ymed)^2' ) )

distribuicao["Valor X_Nota"] = valor_x_list
distribuicao["Valor Y_Distancia"] = valor_y_list

distribuicao
#distancias["LAT"], distancias["LON"], distancias["KM"] = [0,0,0]

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,1,1,NaN,NaN,NaN,NaN,NaN
1,1,2,NaN,NaN,NaN,NaN,NaN
2,2,3,NaN,NaN,NaN,NaN,NaN
3,1,2,NaN,NaN,NaN,NaN,NaN
4,3,2,NaN,NaN,NaN,NaN,NaN
5,1,2,NaN,NaN,NaN,NaN,NaN
6,2,1,NaN,NaN,NaN,NaN,NaN
7,1,3,NaN,NaN,NaN,NaN,NaN
8,0,3,NaN,NaN,NaN,NaN,NaN
9,2,2,NaN,NaN,NaN,NaN,NaN


## Executando passo 1

### Cálculo do Xmed e Ymed (Valor médio de todos os pontos de X e Y)

In [253]:
xmed =  distribuicao['Valor X_Nota'].mean()
ymed =  distribuicao['Valor Y_Distancia'].mean()

### Cálculo (Xi - Xmed) e (Yi - Ymed) i = índice dos valores de X e Y

In [270]:
dif_X = []
dif_Y = []
prod = []

for i in range(len(alunos_validos)):
    
    #Calcula diferença para cada valor de X e salva numa lista
    difX = valor_x_list[i] - xmed
    
    dif_X.append(difX)
    
    #Calcula diferença para cada valor de Y e salva numa lista
    difY = valor_y_list[i] - ymed
    
    dif_Y.append(difY)
    
    #Calcula produto entre valores
    prod_Difs = difX*difY
    
    prod.append(prod_Difs)
    
#Adiciona na tabela
distribuicao["Xi - Xmed"] = dif_X
distribuicao["Yi - Ymed"] = dif_Y
distribuicao["Prod"] = prod

In [271]:
distribuicao

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,1,1,-0.353774,-0.948113,0.335417,0.125156,0.898919
1,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
2,2,3,0.646226,1.051887,0.679757,0.417609,1.106466
3,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
4,3,2,1.646226,0.051887,0.085417,2.710061,0.002692
5,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
6,2,1,0.646226,-0.948113,-0.612696,0.417609,0.898919
7,1,3,-0.353774,1.051887,-0.372130,0.125156,1.106466
8,0,3,-1.353774,1.051887,-1.424017,1.832703,1.106466
9,2,2,0.646226,0.051887,0.033531,0.417609,0.002692


### Calcula a covariância

In [272]:
passo1 = distribuicao['Prod'].sum()

covXY = passo1/(len(alunos_validos)-1)
covXY

-0.062125547706340024

## Executando passo 2

In [273]:
dif_X_quadrada = []
dif_Y_quadrada = []

for i in range(len(alunos_validos)):
    
    #Eleva cada diferença ao quadrado
    difX_quad = dif_X[i]*dif_X[i]
    
    dif_X_quadrada.append(difX_quad)
    
    #Eleva cada diferença ao quadrado
    difY_quad = dif_Y[i]*dif_Y[i]
    
    dif_Y_quadrada.append(difY_quad)
    
#Adiciona na tabela
distribuicao["(Xi - Xmed)^2"] = dif_X_quadrada
distribuicao["(Yi - Ymed)^2"] = dif_Y_quadrada

In [274]:
distribuicao

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,1,1,-0.353774,-0.948113,0.335417,0.125156,0.898919
1,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
2,2,3,0.646226,1.051887,0.679757,0.417609,1.106466
3,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
4,3,2,1.646226,0.051887,0.085417,2.710061,0.002692
5,1,2,-0.353774,0.051887,-0.018356,0.125156,0.002692
6,2,1,0.646226,-0.948113,-0.612696,0.417609,0.898919
7,1,3,-0.353774,1.051887,-0.372130,0.125156,1.106466
8,0,3,-1.353774,1.051887,-1.424017,1.832703,1.106466
9,2,2,0.646226,0.051887,0.033531,0.417609,0.002692


### Calcula Variância de X e Y

In [278]:
passo21 = distribuicao['(Xi - Xmed)^2'].sum()
passo22 = distribuicao['(Yi - Ymed)^2'].sum()

#Variância de X
varX = passo21/(len(alunos_validos)-1)

#Variância de Y
varY = passo22/(len(alunos_validos)-1)


In [279]:
varX

0.7131136546543854

In [280]:
varY

0.8551149065545914

## Executar passo 3

### Calcular Correlação

In [281]:
from math import sqrt

desvio_padraoX = sqrt(varX)
desvio_padraoY = sqrt(varY)

#Calculo da correlação
corrXY = covXY/(desvio_padraoX*desvio_padraoY)

corrXY

-0.07955703988423876